### Effective Python, 59 ways specific ways to write better python
by Brett Slatkin

### Chapter 1

#### Item 7: Use List Comprehensions Instead of <code>map</code> and <code>filter</code>

_list comprehensions_ provide a compact syntax for deriving one list from another.

In [33]:
a = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
squares = [x**2 for x in a]
print(squares)

[1, 4, 9, 16, 25, 36, 49, 64, 81, 100]


These are clearer than <code>map</code> built-in which would require a <code>lambda</code>expression, which can be visually noisy.  However they do not accept arguments; for single argument fuctions, use <code>map</code> with a <code>lambda</code> expression.

In [34]:
squares = map(lambda x: x ** 2, a)
print(squares)

List comprehensions can also accept filters.

In [35]:
even_squares = [x**2 for x in a if x % 2 == 0]
print(even_squares)

[4, 16, 36, 64, 100]


The <code>filter</code> built-in funcion can be used with <code>map</code> to achieve the same outcome, but is visually noisier.

In [36]:
a

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [37]:
alt = map(lambda x: x**2, filter(lambda x: x % 2 == 0, a))
assert even_squares == list(alt)
even_squares == list(alt)

False

Dictionaries and set have their own equivalents of list comprehensions. These make it easy to create derivative data structures when writing algorithms.

In [38]:
chile_ranks = {'ghost': 1, 'habanero': 2, 'cayenne': 3}

In [39]:
rank_dict = {rank: name for name, rank in chile_ranks.items()}
chile_len_set = {len(name) for name in rank_dict.values()}
print(rank_dict)
print(chile_len_set)

{1: 'ghost', 2: 'habanero', 3: 'cayenne'}
{8, 5, 7}


- List comprehensions are clearer than the <code>map</code> and <code>filter</code> built-in functions because they don't require extra <code>lambda</code>expressions.
- List comprehensions allow you to easily skip items from the input list, a behavior <code>map</code> doesn't support without help from <code>filter</code>.
- Dictionaries and sets also support comprehension expressions.

---

#### Item 8: Avoid More Than Two Expressions in List Comprehensions

List comprehensions support multiple levels of looping. The loops run in order from left to right.

In [3]:
matrix = [[1, 2, 3], [4, 5, 6], [7, 8, 9]]
flat = [x for row in matrix for x in row]
print(flat)

[1, 2, 3, 4, 5, 6, 7, 8, 9]


This is a simple example and is still very readable and a reasonable use of multiple loops.  We can replicate the two-level deep layout of the input list to create a two-dimensional matrix.

In [4]:
squared = [[x**2 for x in row] for row in matrix]
print(squared)

[[1, 4, 9], [16, 25, 36], [49, 64, 81]]


This would be about the limit of readability.  If another loop was included, this list comprehension would need to be split over multiple lines.

In [7]:
my_lists = [
    [[1, 2, 3], [4, 5, 6]],[[7, 8, 9], [10, 11, 12]],
]

flat = [x for sublist1 in my_lists
        for sublist2 in sublist1
        for x in sublist2]
print(flat)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]


At this point, the multiline comprehension isn't much shorter than the alternative.
The same result can be produced using a regular nested <code>for</code> loop.  This is equivalent to the previous example.

In [9]:
flat = []
for sublist1 in my_lists:
    for sublist2 in sublist1:
        flat.extend(sublist2)

print(flat)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]


List comprehensions also support multiple <code>if</code> conditions.

In [14]:
a = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
b = [x for x in a if x > 4 if x % 2 == 0]
c = [x for x in a if x > 4 and x % 2 == 0]
print(a)
print(b)
print(c)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
[6, 8, 10]
[6, 8, 10]


Conditions can be specified at each level of looping after the <code>for</code> expression.
Filter a matrix to only pass values divisible by 3 in rows that sum to 10 or higher.
This will short but difficult to read.

In [15]:
matrix = [[1, 2, 3], [4, 5, 6], [7, 8, 9]]
filtered = [[x for x in row if x % 3 == 0]
             for row in matrix if sum(row) >= 10]
print(filtered)

[[6], [9]]


The rule of thumb is to avoid using more than two expressions in a list comprehension. This could to two conditions, two loops, or one condition and one loop.  If it's more complicated than this, a normal <code>if</code> and <code>for</code> would be a better solution. Additionally, a helper function might be in order.

- List comprehensions support multiple levels lf loops and multiple conditions per loop level.
- List comprehensions with more than two expressions are very difficult to read and should be avoided.

---

#### Item 9: Concider Generator Expressions for Large Comprehensions

List comprehensions that produce large output lists can consume significant amounts of memory and cause the program to crash.

In [17]:
value = [len(x) for x in open('my_file.txt')]
print(value)

[80, 75, 80, 79, 75, 8]


To solve this, Python provides _generator expressions_, a generalization of list comprehensions and generators.
A generator expression produce an iterator object that yields one item at a time form the expression.

In [18]:
it = (len(x) for x in open('my_file.txt'))
print(it)

<generator object <genexpr> at 0x103c206d0>


The generator iterator object at amemory location.

The returned iterator  can be advanced one step at a time to produce the next output from the generator expression, using the <code>next</code>built-in function.
The generator can be consumed without risk of a blowup in memory usage.

In [20]:
print(next(it))
print(next(it))

80
75


Generator expressions can be composed together.  Here the output of the previous expression is used as the input of the next expression.

In [21]:
roots = ((x, x**0.5) for x in it)

In [22]:
print(next(roots))

(80, 8.94427190999916)


Each time the iterator is advanced, it will also advance  the interior iterator, creating a domino effect of looping, evaluating conditional expressions, and passing around inputs and outputs.
It must be remembered that iterators returned by generators are stateful and are consumed.  Therefore they can only be used once.

- List comprehensions can cause problems for large inputs by using too much memory.
- Generator expressions avoid memory issues by producing outputs one at a time as an iterator.
- Generator expressions can be composed by passing the iterator from one generator expression into the <code>for</code> subexpression of another.
- Generator expressions execute very quickly when chained together.